# Arbres de décision en Python: cas de la classification

<table><tr>
<td> <img src="files/figures/DT_1.png" width="300px"/> </td>
<td> </td>
<td> </td>
<td> <img src="files/figures/DT_2.png" width="300px"/> </td>
</tr></table>

## Librairies

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Loader les data

In [16]:
data = pd.read_csv("./data/iris.csv")

In [17]:
data

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


Nous allons prédire la variable `variety` en fonction des variables `sepal.length`, 	`sepal.width`, `petal.length`, `petal.width`.

## Arbre de décision dans un contexte de classification

La classe `DecisionNode` ci-dessous représente un **noeud** d'un **arbre de décision**, ou encore, un **arbre de décision** en soi. 

*En effet, en programmation, un objet "noeud" qui pointe vers ses fils est une structure de données suffisante pour représenter un arbre binaire.*

- Un **noeud interne** représente un *split* de la forme $x_k \leq s$ (où $x_k$ est la $k$-ième composante de $\mathbf{x}$).<br>
Chaque noeud interne donc est associé à un `feature_index` $k$ et un `threshold` $s$.<br>
Un noeud interne est également associé à `gain`: le gain informationnel (défini en terme d'entropie...) engendré par son split.
- Une **feuille** représente un sous-ensemble des data $R_m$ (*région*, *cellule de partition*).<br>
Chaque feuille est donc associée une `value`: la target qui apparaît le plus souvent dans les data dans $R_m$:<br>

$$\hat{y}_{R_m} = \arg \max \{ \mathrm{freq}_{R_m}(y_i) : \mathbf{x_i} \in R_m \}$$

In [18]:
class DecisionNode():
    """
    Implements a decision node, or quivalently, a decision tree.
    
    As usual, a binary tree is identified with a root node contaning as chidren
    a left subtree and a right subtree.
    
    Here, an *internal node* represents a split of the form "x_m <= s",
    where m is the `feature_index` and s the `thresold` of the split.
    
    A *leaf node* represents a subset of the data (region, box, partition cell).
    It is associated with a `value`: the most fequent target of the data in this region.
    """
    
    def __init__(self, feature_index=None, threshold=None, 
                 left=None, right=None, gain=None, value=None):
        """
        Constructor.
        
        Parameters
        ----------
        feature_index : int
            index i of variable x_m for the node split (x_m <= s).
        threshold : Union[int, float]
            threshold s for the node split (x_m <= s).
        left : Union[DecisionNode, None]
            Left child of the node
        right : Union[DecisionNode, None]
            Right child of the node
        gain : float
            information gain induced by the node split (x_m <= s).
        value : Union[float, None]
            if the node is a leaf, then value of this leaf.
        """

        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.gain = gain

        # for leaf node
        self.value = value
        
        # children
        self.left = left
        self.right = right

    def print_tree(self, tab=0):
        """Prints the node (i.e., the tree)."""
        
        if self.left is None and self.right is None: # leaf
            
            print("\t"*(tab), f"{self.value:}")
            
        else:                                        # internal node
            self.left.print_tree(tab+1)
            print("\t"*(tab), f"X_{self.feature_index} < {self.threshold:.2f}")
            self.right.print_tree(tab+1)

La classe `DecisionTreeClassifier` représente un **arbre de décision** dans un contexte de classification.

In [29]:
class DecisionTreeClassifier():
    """
    Implements a decision tree for classification.
    """
    
    def __init__(self, min_samples=2, max_depth=2):
        """
        Constructor.
        
        Parameters
        ----------
        min_samples : int
            Minimal number of data necessary to envision a split.
            Otherwise, the node is a leaf (subset of data).
        max_depth : int
            Maximal depth of the tree.
            When max_depth is reached, no further split is performed.
        """
        
        # the root will become the decision tree
        self.root = None
        
        # stopping conditions
        self.min_samples = min_samples
        self.max_depth = max_depth
        
    
    def split_data(self, dataset, split):
        """
        Splits dataset according to a given split of the form x_m <= s.
        
        Parameters
        ----------
        dataset : ndarray
            Dataset to be split
        split : tuple
            A split "x_m <= s" is rerpresented by the tuple (m, s),
            where m is the feature_index and s the threshold.
        
        Returns
        -------
        dataset_left, dataset_right : tuple[ndarray, ndarray]
            dataset_left: data satisfying the condition x_m <= s.
            dataset_right: data satisfying the scondition x_m > s.
        """
        
        pass
    
    
    def entropy(self, y):
        """
        Computes the entropy of a set of targets y.
        
        Parameters
        ----------
        y : ndarray
            Target column of the dataset.
        
        Returns
        -------
        entropy : float
            Entropy of the targets y.
        """
        
        pass
    
    
    def information_gain(self, dataset, dataset_left, dataset_right):
        """
        Computes the information gain induced by a split.
        Information gain can be measued in two ways:
        *Gini index* or *entropy* of the targets.
        
        Suppose that a split of a dataset has induces the 2 datasets:
        dataset_left and dataset_right (cf. method split_data).
        This function computes the entropy $ent_1$ of the y's of dataset as well as
        the weighted entropy $ent_2$ of the y's of left_dataset and right_dataset.
        The information gain is then given by $ent_1 - ent_2$.
        
        Parameters
        ----------
        dataset : ndarray
            Dataset before split.
        dataset_left : ndarray
            First dataset induced by the split.
        dataset_right : ndarray
            Second dataset induced by the split.
        
        Returns
        -------
        gain : float
            information gain (i.e., entropy reduction) induced by the split.
        """
        
        pass

    
    def best_split(self, dataset):
        """
        Computes the best split for a dataset.
        
        For all feature $m$ and and all possible value $s$ of that feature,
        split the dataset according to the condition "x_m <= s" (self.split_data(...)).
        The split "x_m <= s" generates two datatsets dataset_left and dataset_right.
        Compute the information gain associated to the three datasets (self.information_gain).
        Select the split associated with the largest information gain.
        
        Parameters
        ----------
        dataset : ndarray
            Dataset before split.
        
        Returns
        -------
        best_split : dict
            dictionary to store the best split.
            The keys of the dictionare are: 
            "feature_index", "threshold", "dataset_left", "dataset_right", "gain".
        """
        
        pass
    
    
    def build_tree(self, dataset, depth=0):
        """
        Builds the decision tree recursively.
        
        Parameters
        ----------
        dataset : ndarray
            Initial dataset to be split by the decision tree.
        depth : int
            Depth of the decision tree that has been built so far.
        
        Returns
        -------
        DecisionNode : DecisionNode
            The decision tree that represent the best successive splits
            for to the dataset.
        """
        
        pass
    
    
    def fit(self, X, Y):
        """
        Fits the decision tree on the features X and targets Y.
        
        Parameters
        ----------
        X : ndarray
            Feature columns of the dataset.
        Y : ndarray
            Target column of the dataset.
        """
        
        pass
        
        
    def predict_single_point(self, x, tree):
        """        
        Predict the target y_hat associated to a point x.
        
        Parameters
        ----------
        x : ndarray
            Point x = (x_1,...,x_M) whose target is to be predicted.
            
        Returns
        -------
        y_hat : float
            Target associated to x.
        """
        
        pass
        
    
    def predict(self, X):
        """
        Predict the targets associated to a set of points.
        
        Parameters
        ----------
        X : ndarray
            Tensor of points X (dim N x M) whose targets are to be predicted.
            
        Returns
        -------
        Y : ndarray
            Tensor of targets (dim N x 1) associated to X.
        """
        
        pass

### Exercice 1

Complétez la méthode `split_data(...)` qui, étant donné un dataset `dataset` split `split` de la forme $x_m \leq s$, retourne deux datasets qui correspondent aux lignes de `dataset` qui satisfont et ne satisfont pas la condition `split`, respectivement.

Testez votre méthode comme suit:
```
tree = DecisionTreeClassifier()
dataset = np.array(data)
split = (2, 0.1) # split x_2 <= 0.1
dataset_left, dataset_right = tree.split_data(dataset, split)
dataset_left.shape, dataset_right.shape
```

### Exercice 2

Complétez la méthode `information_gain(...)` qui, étant donné trois datasets `dataset`, `dataset_left` et `dataset_right`, retourne le **gain informationnel** associé aux targets $y_i$ de `dataset` et de `dataset_left` et `dataset_right`. 

Ici, le gain informationnel correspond à la **réduction d'entropie**. Le calcul de l'entropie d'un ensemble de targets `y` est obtenu par la méthode `entropy(...)` déjà implémentée. Le **gain informationnel** est alors calculé comme suit:

\begin{eqnarray*}
entropy   & = & \mathrm{Entropy} \left\{ y_i : i \in \mathrm{dataset} \right\} \\
&& \\
entropy_l & = & \frac{|\mathrm{dataset\_left}|}{|\mathrm{dataset}|} \cdot \mathrm{Entropy} \left\{ y_i : i \in \mathrm{dataset\_left} \right\} \\
&& \\
entropy_r & = & \frac{|\mathrm{dataset\_right}|}{|\mathrm{dataset}|} \cdot \mathrm{Entropy} \left\{ y_i : i \in \mathrm{dataset\_right} \right\} \\
&& \\
gain & = & entropy - (entropy_l + entropy_r)
\end{eqnarray*}

où $|X|$ dénote le nombre d'éléments de l'ensemble $X$.

Tester votre méthode comme suit:
```
tree = DecisionTreeClassifier()
dataset = np.array(data)
split = (2, 1.5) # split x_2 <= 0.1
dataset_left, dataset_right = tree.split_data(dataset, split)
tree.information_gain(dataset, dataset_left, dataset_right)
```

### Exercice 3

Complétez la méthode `best_split(...)` qui, étant donné un dataset `dataset`, retourne le split induisant le plus grand gain informationnel. Ce best split est obtenu par l'algorithme suivant:

<img src="files/figures/DT_algo_3.png" width="650px"/>

La variable `best_split` sera un dictionnaire avec les clés suivantes:
```
"feature_index"
"threshold"
"dataset_left"
"dataset_right"
"gain"
```

Tester votre méthode comme suit:
```
tree = DecisionTreeClassifier()
dataset = np.array(data)
best_split = tree.best_split(dataset)
```

### Exercice 4

Complétez la méthode `leaf_value(...)` qui, étant donné un sous-ensemble du dataset `region`, retourne la target qui apparaît le plus fréquemment dans ce sous-dataset. On rappelle que chaque feuille de l'arbre de décision est associée avec une région non-splitable du dataset.

Tester votre méthode comme suit:
```
tree = DecisionTreeClassifier()
dataset = np.array(data)
best_split = tree.best_split(dataset)
region = best_split['dataset_right']
tree.leaf_value(region)
```

### Exercice 5

Complétez la méthode **récursive** `build_tree(...)` qui, étant donné un dataset `dataset` et une profondeur `depth`, retourne l'arbre de décision correspondant à ce dataset. L'arbre de dlcision est donné par l'algorithme **recursive binary plitting** ci-dessous.

<img src="files/figures/DT_algo_4.png" width="650px"/>

Tester votre méthode comme suit:
```
dataset = np.array(data)
tree = DecisionTreeClassifier(min_samples=3, max_depth=3)
tree.root = tree.build_tree(dataset)
tree.root.print_tree()
```

### Exercice 6

Complétez la méthode `fit(...)` qui correspond à l'entaînement de l'arbre de décision sur les features `X` et les targets `Y` du dataset. Cette méthode contruit l'arbre de décision associée au dataset $(X, Y)$ et assigne ce dernier comme racine de l'arbre.


Tester votre méthode comme suit:
```
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)

classifier = DecisionTreeClassifier(min_samples=3, max_depth=3)
classifier.fit(X_train, Y_train)   # entraînement sur le train set
classifier.root.print_tree()
```

### Exercice 7

Complétez la méthode **récursive** `predict_single_point(...)` qui retourne la prédiction `y_hat` associée au point `x`. Cette méthode fait passer `x` dans les noeuds sucessifs de l'arbre de décision `tree` jusqu'à atteindre une feuille dont la valeur sera la prédiction.

Complétez ensuite la méthode `predict(...)` qui retourne l'enemble des prédictions `X` associée à un ensemble de points `X` (tenseur). Cette méthode appelle la précédent sur chaque ligne de `X`.

Tester votre méthode comme suit:
```
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)

classifier = DecisionTreeClassifier(min_samples=3, max_depth=3)
classifier.fit(X_train, Y_train)   # entraînement sur le train set
classifier.predict_single_point(np.array[0.1, 0.2, 0.3, 0.4. 0.5], classifier.root)
classifier.predict(X_test)
```

### Exercice 8

Exécutez le code ci-dessous pour voir si votre implémentation de la classe ``DecisionTreeClassifier`` fonctionne coorrectement.

#### Train-Test split

In [26]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)

#### Entraînement

In [27]:
classifier = DecisionTreeClassifier(min_samples=3, max_depth=3)
classifier.fit(X_train,Y_train)
# classifier.root.print_tree()

#### Résultats sur le test set

In [3]:
Y_pred = classifier.predict(X_test)
print(classification_report(Y_test, Y_pred))